<a href="https://colab.research.google.com/github/Jasper-Hewitt/Wind_project/blob/main/alternative_finding_noisepollution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#this one uses a different regex, split on 。 and ：
#many of the units of information within the PDF are actually only one sentence so I guess this is fine. 
!pip install fitz
!pip install static
!pip install frontend
!pip install pymupdf --upgrade

In [2]:
# mkdir static
import fitz

doc = fitz.open("dragged_130_146.pdf")

for page in doc:
    text = page.get_text("text")
    with open("output3.txt", "a") as f:
        f.write(text)


In [ ]:
with open('output3.txt', 'r') as file:
    # Read the contents of the file
    contents = file.read()
    # Print the contents
    print(contents)

In [5]:
import re
pattern = "。|:"

split_contents = re.split(pattern, contents)

# print(sentences)


In [ ]:
split_contents

In [7]:
len(split_contents)

128

In [ ]:
# import re
# split_contents = re.split(r'\d+\.', contents)
# split_contents

In [ ]:
!pip install sentence_transformers
#also works with entire alineas 
#below are some other models that don't work. I get a warning signal that says it can't find the model so it switches to MEAN pooling instead. 
#('all-MiniLM-L6-v2'), one of the most popular ones, works but it is an English model 
#("uer/sbert-base-chinese-nli")#('shibing624/text2vec-base-chinese')#('all-MiniLM-L6-v2') 

from sentence_transformers import SentenceTransformer, util
import torch

#alternative model by same creators: ("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"). This one appears to be more popular. Compare results.
# embedder = SentenceTransformer("distiluse-base-multilingual-cased-v1")
embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
# Corpus with example sentences
corpus = split_contents
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['噪音', '水下噪音','聲音']

# Find the closest X sentences of the corpus for each query sentence based on cosine similarity
top_k = min(68, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop X most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))

#calculate accumulated similarity score per keyword

In [9]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [10]:
len(sentence_scores)

94

In [11]:
import pandas as pd
#in case we use more or less keywords, we will have to adjust this code slightly
df = pd.DataFrame([[sentence, scores[queries[0]], scores[queries[1]], 
                    scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]] 
                   for sentence, scores in sentence_scores.items()], 
                  columns=['text', 'similarity_score1', 'similarity_score2', 'similarity_score3', 
                           'accumulated_score'])
df = df.sort_values(by='accumulated_score', ascending=False)

In [21]:
noise_pattern= '遵照辦理'
noise_pattern2='��'
noise_pattern3='�'
df['text'] = df['text'].replace(noise_pattern3,'',regex=True)
df['text'] = df['text'].replace(noise_pattern,'',regex=True)
df['text'] = df['text'].replace(noise_pattern2,'',regex=True)

In [23]:
#delete empty rows
df = df[df['text'].str.len()>6]   
df = df[['text', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score']]
df = df.reset_index(drop=True)
df

,text,similarity_score1,similarity_score2,similarity_score3,accumulated_score
0,\n－ \n－ \n(2)依據噪音振動模擬\n結果顯示，陸上施\n工及風機營運後之\n全頻...,tensor(0.5539),tensor(0.4647),tensor(0.4433),tensor(1.4619)
1,\n二、 水下噪音監測 \n水下噪音監測之目的係為確認打樁期\n間之水下噪音值是否超出值，...,tensor(0.4893),tensor(0.5716),tensor(0.3833),tensor(1.4442)
2,\n3.開發行為屬點狀開\n發 ， 無 大 面 積 施\n工，環境影響說明\n書中已針對施...,tensor(0.3921),tensor(0.3704),tensor(0.3250),tensor(1.0874)
3,\n8.1.2.1 \n8-5 \n(五)施工前不使用聲音驅\n離裝置(ADD),tensor(0.3806),tensor(0.2708),tensor(0.3720),tensor(1.0234)
4,\n，本計畫於施工期間共規劃2種水\n下監測，其目的及規劃內容分別說明如下： \n一、 水...,tensor(0.2692),tensor(0.4633),tensor(0.2864),tensor(1.0189)
...,...,...,...,...,...
86,\n另本開發集團沃旭能源公司已於107年\n2月7日宣布在彰化建立亞洲首個百萬\n瓦等級儲...,tensor(-0.0418),tensor(-0.0208),tensor(-0.0679),tensor(-0.1305)
87,\n四、 創造在地的商機 \n未來將在彰化投資必須的項目、如產\n業人力需求、運輸，以在地...,tensor(-0.0703),tensor(-0.0794),tensor(-0.0152),tensor(-0.1650)
88,\n三、 在地人才培訓 \n(一) 提供綠能全額獎學金(8名)保留彰\n化子名額，預計10...,tensor(-0.0871),tensor(-0.1202),tensor(-0.0429),tensor(-0.2502)
89,以上這些皆為本集團公司\n自願性的企業社會責任舉措,tensor(-0.0999),tensor(-0.1250),tensor(-0.0614),tensor(-0.2863)


## determine threshold

## Summary with GPT 

In [ ]:
#I think we can just give it a bunch of sentences and let it write a summary of the specs that are mentioned. If this does not work we will have to try to find a way to put the sentences in the dataframe above 
#back into the original order. We would have to assign a sentence number to all of the sentences when we split it. 


In [41]:
#try to find a way to copy all of the text form the df and put it in a variable with the type string, this will make it easier to copy paste it into the GPT API. 
sbert_out=df.head(12)
sbert_out=sbert_out['text']
sbert_out=sbert_out.to_frame()
sbert_out

,text
0,\n－ \n－ \n(2)依據噪音振動模擬\n結果顯示，陸上施\n工及風機營運後之\n全頻...
1,\n二、 水下噪音監測 \n水下噪音監測之目的係為確認打樁期\n間之水下噪音值是否超出值，...
2,\n3.開發行為屬點狀開\n發 ， 無 大 面 積 施\n工，環境影響說明\n書中已針對施...
3,\n8.1.2.1 \n8-5 \n(五)施工前不使用聲音驅\n離裝置(ADD)
4,\n，本計畫於施工期間共規劃2種水\n下監測，其目的及規劃內容分別說明如下： \n一、 水...
5,\n，「不使用聲音驅離裝置(ADD)\n」之承諾，將納入環境影響說明書8.1.2.1節\n...
6,現依\n第327次環境影響評估審查委員會決議\n內容修正為「施工期間於距離打樁位\n8.1....
7,此項承諾之相關文字內容已載\n於環說報告書8.1.2.1節(P8-6~7)，並與\n環說報告...
8,\n8.1.1 \n8-1 \n(二)施工期間於距離打樁\n位置外750公尺處選擇\n合理...
9,\n(二) 協助指導學生離岸風機相關的論\n文


In [43]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 KB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.1-py3-none-any.whl size=67316 sha256=7881e4b6dd0c2ad2b676452782ec4c91015ef7a47d02ce4f697a92daad13b5ca
  Stored in directory: /root/.cache/pip/wheels/2f/9c/55/95d3609ccfc463eeffb96d50c756f1f1899453b85e92021a0a
Successfully built openai


In [45]:
#there is one problem with chatgpt and that is that it only allows us to insert 4000 tokens. so we either have to be super selective with the information we put in, or 
#put it in in batches. 
#I think it counts whitespaces as tokens. because when I copy paste this in word it says it is only 923 words. try a regex that deletes whitespaces ands see what happens.
import os
import openai
import json

openai.api_key = '' #remove key when uploading to github
summary = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt="""幫我寫總結，強調關於噪音的規格，請你使用比較簡單的語言。文本：0	－ － (2)依據噪音振動模擬 結果顯示，陸上施 工及風機營運後之 全頻及低頻噪音， 經與實測背景值合 成之後，各敏感受 體皆可符合環境音 量標準，噪音增量 屬無影響或可忽略 影響
1	二、 水下噪音監測 水下噪音監測之目的係為確認打樁期間之水下噪音值是否超出值，故本 計畫承諾於每支基礎施工時，均於警 戒區周界(750m處)執行一次打樁噪音 監測，每次監測時將從緩啟動起開始 ，打樁全程均將監測並全程使用減噪 工法
2	3.開發行為屬點狀開發，無大面積施工，環境影響說明 書中已針對施工及 營運期間之「地形 及地質(含海岸地形 變 遷 影 響 分 析 ) 」 「 水 文 及 水 質 」 「空氣品質」「噪 音 振 動 ( 含 水 下 噪 音)」「風機基礎 影響」「陸域電 磁場」「廢棄物」 「剩餘土石方處理 計 畫 」 「 通 訊 干 擾」「溫室氣體減 量」「生態環境(含 陸域、海域、魚類
3	8.1.2.1 8-5 (五)施工前不使用聲音驅 離裝置(ADD)
4	，本計畫於施工期間共規劃2種水 下監測，其目的及規劃內容分別說明如下： 一、 水下聲學監測 水下聲學監測目的係為打樁期間針對 是否有鯨豚靠近施工區域而進行之監 測，本計畫原規劃內容為「施工期間 於距打樁位置750m處及1500m處各放 置2個被動式鯨豚聲音偵測器，以持續 監測是否有鯨豚在附近活動」
5	，「不使用聲音驅離裝置(ADD) 」之承諾，將納入環境影響說明書8.1.2.1節 「一、鯨豚」之「(二) 施工期間時的監測 及預防對策」中
6	現依 第327次環境影響評估審查委員會決議 內容修正為「施工期間於距離打樁位 8.1.2.1 8.2.2 8-4 8-6~7 8-26 10 審查意見 覆說明 修訂處 章節 頁次 置外750公尺處選擇合理方位設置4座 水下聲學監測設施並分布於4個方位， 持續監測打樁水下噪音值及是否有鯨 豚在附近活動」
7	此項承諾之相關文字內容已載 於環說報告書8.1.2.1節(P8-6~7)，並與 環說報告書表8.2.2-2「施工階段環境 監測計畫表」中「水下噪音」之第一 項監測內容相符，如表8.2.2-2所示， 請參閱
8	8.1.1 8-1 (二)施工期間於距離打樁 位置外750公尺處選擇 合理方位設置4座水下 聲學監測設施並分布 於4個方位，持續監測 打樁水下噪音值
9	(二) 協助指導學生離岸風機相關的論 文
10	－ － (4)鯨豚：4案風場均 非位於中華白海豚 野生動物重要棲息 環境預告範圍，並 依水下噪音模擬評 估結果，已擬定海 豚保護措施
11	，打樁工程應採緩啟動(softstart) 持續至少30分鐘，由低力道的打樁慢慢漸 進到全力道的打樁，讓鯨豚類仍有時間離 開打樁噪音源""", #insert promt for example:幫我寫總結，強調關於噪音的規格，請你使用比較簡單的語言/
  temperature=0.9,
  max_tokens=4000,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary = summary + text

summary

InvalidRequestError: ignored